In [1]:
import torch
import torch.nn as nn
from sinabs.backend.dynapcnn import DynapcnnNetwork
from sinabs.backend.dynapcnn import DVSLayer
from sinabs.layers import Merge, IAFSqueeze, SumPool2d
from sinabs.activation.surrogate_gradient_fn import PeriodicExponential
import sinabs.layers as sl

from torch.nn import CrossEntropyLoss
from torch.optim import Adam

from tonic.datasets.nmnist import NMNIST
from tonic.transforms import ToFrame
from torch.utils.data import DataLoader
import numpy as np
from tqdm.notebook import tqdm
from statistics import mode

device = torch.device('cpu')
torch.manual_seed(0)

/home/samurai2077/anaconda3/envs/speck-rescnn/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
channels = 1
height = 34
width = 34
batch_size = 1

input_shape = (channels, height, width)

In [3]:
class SNN(nn.Module):
    def __init__(self, input_shape) -> None:
        super().__init__()

        self.dvs = DVSLayer(input_shape=(input_shape[1], input_shape[2]))
        # -- chip core A --
        self.conv1 = nn.Conv2d(1, 10, 2, 1, bias=False)
        self.iaf1 = IAFSqueeze(batch_size=batch_size, min_v_mem=-1.0, spike_threshold=1.0, surrogate_grad_fn=PeriodicExponential())
        self.pool1 = nn.AvgPool2d(2,2)
        # -- chip core B --
        self.conv2 = nn.Conv2d(10, 10, 4, 1, bias=False)
        self.iaf2 = IAFSqueeze(batch_size=batch_size, min_v_mem=-1.0, spike_threshold=1.0, surrogate_grad_fn=PeriodicExponential())
        # -- chip core C --
        self.conv3 = nn.Conv2d(10, 1, 2, 1, bias=False)
        self.iaf3 = IAFSqueeze(batch_size=batch_size, min_v_mem=-1.0, spike_threshold=1.0, surrogate_grad_fn=PeriodicExponential())
        # -- chip core D --
        self.fc1 = nn.Linear(144, 200, bias=False)
        self.iaf4 = IAFSqueeze(batch_size=batch_size, min_v_mem=-1.0, spike_threshold=1.0, surrogate_grad_fn=PeriodicExponential())
        # -- chip core E --
        self.fc2 = nn.Linear(200, 10, bias=False)
        self.iaf5 = IAFSqueeze(batch_size=batch_size, min_v_mem=-1.0, spike_threshold=1.0, surrogate_grad_fn=PeriodicExponential())

        # -- layers ignored during deployment --
        self.flat = nn.Flatten()

    def init_weights(self):
        for name, layer in self.named_modules():
            if isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear):
                nn.init.xavier_normal_(layer.weight.data)

    def detach_neuron_states(self):
        for name, layer in self.named_modules():
            if name != '':
                if isinstance(layer, sl.StatefulLayer):
                    for name, buffer in layer.named_buffers():
                        buffer.detach_()

    def forward(self, x):
        
        dvs_out = self.dvs(x) # 0
        
        con1_out = self.conv1(dvs_out) # 4
        iaf1_out = self.iaf1(con1_out) # 5
        pool1_out = self.pool1(iaf1_out) # 6

        conv2_out = self.conv2(pool1_out) # 7
        iaf2_out = self.iaf2(conv2_out) # 8

        conv3_out = self.conv3(iaf2_out) # 9
        iaf3_out = self.iaf3(conv3_out) # 10

        flat_out = self.flat(iaf3_out) # 15
        
        fc1_out = self.fc1(flat_out) # 11
        iaf4_out = self.iaf4(fc1_out) # 12
        fc2_out = self.fc2(iaf4_out) # 13
        iaf5_out = self.iaf5(fc2_out) # 14

        return iaf5_out
    
snn = SNN(input_shape)
snn.init_weights()
snn.to(device)

SNN(
  (dvs): DVSLayer(
    (pool_layer): SumPool2d(norm_type=1, kernel_size=(1, 1), stride=None, ceil_mode=False)
    (crop_layer): Crop2d((0, 34), (0, 34))
    (flip_layer): FlipDims()
  )
  (conv1): Conv2d(1, 10, kernel_size=(2, 2), stride=(1, 1), bias=False)
  (iaf1): IAFSqueeze(spike_threshold=Parameter containing:
  tensor(1.), min_v_mem=Parameter containing:
  tensor(-1.), batch_size=1, num_timesteps=-1)
  (pool1): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (conv2): Conv2d(10, 10, kernel_size=(4, 4), stride=(1, 1), bias=False)
  (iaf2): IAFSqueeze(spike_threshold=Parameter containing:
  tensor(1.), min_v_mem=Parameter containing:
  tensor(-1.), batch_size=1, num_timesteps=-1)
  (conv3): Conv2d(10, 1, kernel_size=(2, 2), stride=(1, 1), bias=False)
  (iaf3): IAFSqueeze(spike_threshold=Parameter containing:
  tensor(1.), min_v_mem=Parameter containing:
  tensor(-1.), batch_size=1, num_timesteps=-1)
  (fc1): Linear(in_features=144, out_features=200, bias=False)
  (iaf4): IAFSqu

In [4]:
hw_model = DynapcnnNetwork(
    snn=snn,
    input_shape=input_shape,
    batch_size=batch_size,
    discretize=True
)

In [5]:
input_dummy = torch.randn((batch_size, *input_shape))

In [6]:
out = hw_model(input_dummy)

In [7]:
print(out)

tensor([[[[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]]]], grad_fn=<PowBackward0>)
